Creating MEDS files
https://github.com/esheldon/meds/wiki/Creating-MEDS-Files-in-Python

In [1]:
import numpy as np
import pandas as pd

import sklearn.neighbors as neighbors

from importlib import reload
import copy
import synthetic.render.generator as gen

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import multiprocessing as mp

import sys
import os
import math
import logging
import time
import galsim
import fitsio as fio

import ngmix

import synthetic.render.render as render
import synthetic.render.icl as icl
import synthetic.render.frame as frame

import images
import meds
import subprocess

In [2]:
catalogs = []
catalog_tag = '/e/ocean1/users/vargatn/DES/SIM_DATA/SKY_MODEL_EPSILON/res_clust_v2_z0l0/epsilon_clust_v8-z0l0_15000_{:03d}.fits'
for i in np.arange(1):
    fname = catalog_tag.format(i)
    data = fio.read(fname)
    catalog_cluster = data[data['LOC'] == 1]
    catalog_field = data[data['LOC'] == 0]
    catalog = np.hstack((catalog_cluster, catalog_field))
    catalog['G1'] = 0.3
    catalog['G2'] = -0.2
    catalogs.append(catalog)

In [5]:

i = 0
name = 'canvas_{:03d}'.format(i)
print(name)
fr = frame.Frame(catalog, name=name, noise_std=4)
fr.render()
fr.extract()
fr.ksb()


canvas_000
starting xshear calculations in 100 processes


In [15]:
fr.scat.shape

(9323,)

In [8]:
meds.util.get_meds_input_struct(10)

array([(-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.),
       (-9999, -9999, -9999., -9999.), (-9999, -9999, -9999., -9999.)],
      dtype=[('id', '<i8'), ('box_size', '<i8'), ('ra', '<f8'), ('dec', '<f8')])

In [7]:
meds.util.get_meds_output_struct(10, 1, )

array([(-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-9999], [-9999.], [-9999.], [-9999], [-9999], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.], [-9999.]),
       (-9999, -9999, -9999., -9999., 0, [-9999], [-

In [17]:
"""
column             type     description
----------------------------------------------------------------------
id                 i8       The id column from the input file
ncutout            i8       number of cutouts for this object
box_size           i8       box size for each cutout
file_id            i8[NMAX] zero-offset id into the file names in the 
                            image_info extension
start_row          i8[NMAX] zero-offset, start of cutout in the 
                            image extensions.
orig_row           f8[NMAX] zero-offset position in original image      FLOAT the position within the image EXACT
orig_col           f8[NMAX] zero-offset position in original image      FLOAT the position within the image EXACT
orig_start_row     i8[NMAX] zero-offset start corner in original image  INT This is the starting corner in row
orig_start_col     i8[NMAX] zero-offset start corner in original image  INT This is the starting corenr is col
cutout_row         f8[NMAX] zero-offset position in cutout imag         FLOAT the position within the cutout EXACT
cutout_col         f8[NMAX] zero-offset position in cutout image        FLOAT the position within the cutout EXACT
dudrow             f8[NMAX] jacobian of transformation                      
dudcol             f8[NMAX] row,col->ra,dec tangent plane (u,v)
dvdrow             f8[NMAX] Note Source Extractor uses y for row
dvdcol             f8[NMAX]
"""

'\ncolumn             type     description\n----------------------------------------------------------------------\nid                 i8       The id column from the input file\nncutout            i8       number of cutouts for this object\nbox_size           i8       box size for each cutout\nfile_id            i8[NMAX] zero-offset id into the file names in the \n                            image_info extension\nstart_row          i8[NMAX] zero-offset, start of cutout in the \n                            image extensions.\norig_row           f8[NMAX] zero-offset position in original image\norig_col           f8[NMAX] zero-offset position in original image\norig_start_row     i8[NMAX] zero-offset start corner in original image\norig_start_col     i8[NMAX] zero-offset start corner in original image\ncutout_row         f8[NMAX] zero-offset position in cutout imag\ncutout_col         f8[NMAX] zero-offset position in cutout image\ndudrow             f8[NMAX] jacobian of transformation \nd